In [20]:
import pandas as pd

### Code which puts survey into flat format

In [21]:
#dairy flat file
dff = pd.DataFrame()

### ID mappings

In [ ]:
id_mappings = {
    "grazing_quality": {
        "HIGH": 1,
        "LOW": 2
    }
}

### Make checks to see if inputs match options


In [ ]:
units = {
    "fertilizer_application_rate": 12, # t/ha
    "feed_weight": 9, # t
}

# make dry matter conversions

In [27]:
from config_loader import load_toml

feed_items = load_toml("feed.toml")["feed"]
herd_sections = load_toml("herd.toml")["herd_section"]
fertilizers = load_toml("fertilizer.toml")["fertilzier"]

In [24]:
feed_components_input = []

for feed in feed_items:
    print(feed["cft_name"])
    for hs in herd_sections:
        feed_components_input.append({
            "item": feed["cft_id"],
            "region": feed["region_name"],
            "herd_section": hs,
            "dry_matter": {
                "value": dff[f"feed.{feed["cft_name"]}.{hs["cft_name"]}.tonne"],
                "unit": units["feed_weight"]
            },
            "certified": False
        })

print(feed_components_input)

grass_fresh_managed


KeyError: 'feed.grass_fresh_managed.calf_dairy.tonne'

### Code which loads flat table into JSON format

In [ ]:


farm_input = {
    "country": "Poland", 
    "territory": None,
    "climate": "Cool Temperate Moist",
    "average_temperature": {"value": 10, "unit": "°C"},
    "latitude": 52.679,
    "longitude": 20.030,
    "soil_characteristics": "Sandy Soils",
    "farm_identifier": dff["farm_id"]
}

general_input = {
    "grazing_area": {"value": dff["general.grazing_area"], "unit": "ha"},
    "feed_approach": 1, # 1 = dmi
    "fertilizer_approach": 2, # 2 = Grazing, grass silage and hay area combined
}

milk_production_input = {
    "variety": dff["main_breed_variety"],
    "reporting_year": dff["milk_year"],
    "date_time": "start",
    "date_month": 1, # Season always starts in Jan
    "name": dff["farm_id"] + str("_") + dff["milk_year"].astype(str),
    "product_dry": {"value": dff["total_milk_production_litres"], "unit": "litres"},
    "fat_content": dff["milk_fat_content_percent"],
    "protein_content": dff["milk_protein_content_percent"],
    "protein_measure": 1 # 1 = true protein
}

herd_sections_input = [
    {
        "phase": "calf_dairy",
        "animals": dff["calf_dairy.herd_count"],
        "live_weight": {
            "value": dff["calf_dairy.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["calf_dairy.sold_count"],
        "sold_weight": {
            "value": dff["calf_dairy.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["calf_dairy.purchased_count"],
        "purchased_weight": {
            "value": dff["calf_dairy.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "heifer",
        "animals": dff["heifer.herd_count"],
        "live_weight": {
            "value": dff["heifer.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["heifer.sold_count"],
        "sold_weight": {
            "value": dff["heifer.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["heifer.purchased_count"],
        "purchased_weight": {
            "value": dff["heifer.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "cow_milk",
        "animals": dff["cow_milk.herd_count"],
        "live_weight": {
            "value": dff["cow_milk.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["cow_milk.sold_count"],
        "sold_weight": {
            "value": dff["cow_milk.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["cow_milk.purchased_count"],
        "purchased_weight": {
            "value": dff["cow_milk.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "cow_dry",
        "animals": dff["cow_dry.herd_count"],
        "live_weight": {
            "value": dff["cow_dry.live_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["cow_dry.sold_count"],
        "sold_weight": {
            "value": dff["cow_dry.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["cow_dry.purchased_count"],
        "purchased_weight": {
            "value": dff["cow_dry.purchased_weight_kg"],
            "unit": "kg"
        }
    }
]

grazing_input = [
    {
        "herd_section": "calf_dairy",
        "days": dff["calf_dairy.grazing_days"],
        "hours": dff["calf_dairy.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["calf_dairy.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "heifer",
        "days": dff["heifer.grazing_days"],
        "hours": dff["heifer.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["heifer.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "cow_milk",
        "days": dff["cow_milk.grazing_days"],
        "hours": dff["cow_milk.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["cow_milk.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "cow_dry",
        "days": dff["cow_dry.grazing_days"],
        "hours": dff["cow_dry.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["cow_dry.grazing_quality"]] # has to be int  1 =  high, 2 = low
    }
]

fertilizers_input = []

for fertilizer in fertilizers:

    # if custom NPK
    if fertilizer["cft_id"] == 44:
        {
            "type": fertilizer["cft_id"], 
            "production": 8, # 8 = Europe 2014
            "custom_ingredients": {
                "n_total_percentage": 6, # 6% as defined by Pawel
                "n_ammonia_percentage": 6, # 6% as defined by Pawel
                "p2o5_percentage": 20, # 20% as defined by Pawel
                "p2o5_percentage_type_id": 4, # 4 = P2O5
                "k2o_percentage": 30, # 30% as defined by Pawel
                "k2o_percentage_type_id": 5 # 5 = K2O 
            },
            "application_rate": {
                "value": dff[f"fertilizers.{fertilizer['key']}.t_per_ha"],
                "unit": units["fertilizer_application_rate"]
            },
            "application_date": "unknown",
            "rate_measure": "product",
            "inhibition": fertilizer["inhibition"] 
        },
    else:
        {
            "type": fertilizer["cft_id"], 
            "production": 8, # 8 = Europe 2014
            "application_rate": {
                "value": dff[f"fertilizers.{fertilizer['key']}.t_per_ha"],
                "unit": units["fertilizer_application_rate"]
            },
            "application_date": "unknown",
            "rate_measure": "product",
            "inhibition": fertilizer["inhibition"]
        }


# =========== Feed components section ============

feed_components_input = []

for feed in feed_items:
    
    for hs in herd_sections:
        feed_components_input.append({
            "item": feed["cft_id"],
            "region": feed["region_name"],
            "herd_section": hs,
            "dry_matter": {
                "value": dff[f"feed.{feed["cft_name"]}.{hs["cft_name"]}.tonne"],
                "unit": units["feed_weight"]
            },
            "certified": False
        })

# =========== Manure section ============

manure_inputs = []

herd_sections = ["calf_dairy", "heifer", "cow_milk", "cow_dry"]

for herd in herd_sections:
    manure_type = dff[f"manure_type.{herd}"] # names of columns
    
    if manure_type != "PIT STORAGE AND SOLID STORAGE":
        manure_inputs.extend([
            {"herd_section": herd, "type": 6, "allocation": 50},  # Pit Storage
            {"herd_section": herd, "type": 1, "allocation": 50},  # Solid Storage
        ])
    else:
        manure_inputs.append({
            "herd_section": herd,
            "type": manure_type,
            "allocation": 100
        })

    

dairy_input = {
    "farm": farm_input,
    "general": general_input,
    "milk_production": milk_production_input,
    "herd_sections": herd_sections_input,
    "grazing": grazing_input,
    "fertilizers": fertilizers_input,
    "feed_components": feed_components_input,
    "feed_additives": [], # leave empty as no additives
    "manure": manure_inputs,
    "direct_energy": [],
    "transport": []
}

KeyError: 'farm_id'